In [7]:
import numpy as np
from typing import Dict, Union
from sturdy.validator.reward import format_allocations


def calc_similarity_matrix(apys_and_allocations: Dict[str, Dict[str, Union[Dict[str, float], float]]],
                               assets_and_pools: Dict[str, Union[Dict[str, float], float]]):
    similarity_matrix = {}
    for miner_a, info_a in apys_and_allocations.items():
        _alloc_a = info_a["allocations"]
        alloc_a = np.array(
            list(format_allocations(_alloc_a, assets_and_pools).values())
        )
        similarity_matrix[miner_a] = {}
        for miner_b, info_b in apys_and_allocations.items():
            if miner_a != miner_b:
                _alloc_b = info_b["allocations"]
                alloc_b = np.array(
                    list(format_allocations(_alloc_b, assets_and_pools).values())
                )
                similarity_matrix[miner_a][miner_b] = np.linalg.norm(alloc_a - alloc_b)
    return similarity_matrix

In [15]:
from sturdy.igreedy import igreedy_allocation_algorithm
from sturdy.pools import generate_assets_and_pools
from sturdy.protocol import AllocateAssets
from sturdy.score import calc_strategy_apy, strategies_apy
from sturdy.validator.reward import calculate_penalties


assets_and_pools = generate_assets_and_pools()
allocation = igreedy_allocation_algorithm(AllocateAssets(assets_and_pools=assets_and_pools))
apy = calc_strategy_apy(allocation, assets_and_pools)

apys_and_allocations = {
    '0': {
        'apy': apy,
        'allocations': allocation
    },
    '1': {
        'apy': apy,
        'allocations': allocation
    }
}
axon_times = {k: 0 for k, _ in apys_and_allocations.items()}

similarity_matrix = calc_similarity_matrix(apys_and_allocations, assets_and_pools)

penalties = calculate_penalties(similarity_matrix, axon_times=axon_times)

In [111]:
from scipy.spatial.distance import pdist, squareform


def generate_sphere_packing(N, M, min_dist):
    # Initialize an empty list to store the valid vectors
    vectors = []

    while len(vectors) < M:
        candidate = np.random.randn(N)  # Generate a random candidate vector
        candidate = candidate / (np.linalg.norm(candidate) * 10)  # Normalize the candidate vector
        # if all(np.linalg.norm(candidate - v) >= min_dist for v in vectors) and np.linalg.norm(candidate) < min_dist + 1e-2:
        if all(np.linalg.norm(candidate - v) >= min_dist for v in vectors):
            vectors.append(candidate)

    return np.array(vectors)

# Example parameters
N = 10  # Dimension
M = 100  # Number of vectors
min_dist = 0.1  # Minimum L2 norm distance

vectors = generate_sphere_packing(N, M, min_dist)

apys_and_allocations = {}
for idx, vec in enumerate(vectors):
    target = vec.copy() + np.array(list(allocation.values()))
    apys_and_allocations[str(idx)] = {
        'allocations': {f'{i}': target[i] for i in range(N)},
    }

axon_times = {k: 0 for k, _ in apys_and_allocations.items()}

similarity_matrix = calc_similarity_matrix(apys_and_allocations, assets_and_pools)

penalties = calculate_penalties(similarity_matrix, axon_times=axon_times)

print(penalties)

{'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 0, '19': 0, '20': 0, '21': 0, '22': 0, '23': 0, '24': 0, '25': 0, '26': 0, '27': 0, '28': 0, '29': 0, '30': 0, '31': 0, '32': 0, '33': 0, '34': 0, '35': 0, '36': 0, '37': 0, '38': 0, '39': 0, '40': 0, '41': 0, '42': 0, '43': 0, '44': 0, '45': 0, '46': 0, '47': 0, '48': 0, '49': 0, '50': 0, '51': 0, '52': 0, '53': 0, '54': 0, '55': 0, '56': 0, '57': 0, '58': 0, '59': 0, '60': 0, '61': 0, '62': 0, '63': 0, '64': 0, '65': 0, '66': 0, '67': 0, '68': 0, '69': 0, '70': 0, '71': 0, '72': 0, '73': 0, '74': 0, '75': 0, '76': 0, '77': 0, '78': 0, '79': 0, '80': 0, '81': 0, '82': 0, '83': 0, '84': 0, '85': 0, '86': 0, '87': 0, '88': 0, '89': 0, '90': 0, '91': 0, '92': 0, '93': 0, '94': 0, '95': 0, '96': 0, '97': 0, '98': 0, '99': 0}


In [118]:
np.linalg.norm(vectors[2])
vectors[0]

array([ 0.00687023, -0.00013409,  0.00755292, -0.0612751 ,  0.05416116,
        0.04018364, -0.00178953, -0.03705576,  0.01185207, -0.00872571])

In [327]:
import numpy as np


def generate_points_with_min_distance(N, dim=3, min_distance=0.1, epsilon=1e-10):
    """
    Generate N points in dim dimensions such that each new point is at least min_distance away from all others.
    """
    points = []

    while len(points) < N:
        new_point = np.random.randn(dim)
        new_point -= np.mean(new_point) + epsilon
        new_point = new_point / np.linalg.norm(new_point) * (min_distance + epsilon)
        # Check if new_point has at least min_distance distance from all existing points
        if all(np.linalg.norm(new_point - existing_point) >= min_distance and np.linalg.norm(new_point - existing_point) < min_distance + epsilon for existing_point in points):
            points.append(new_point)

    return np.array(points)

# Parameters
N = 256  # Number of points to generate
dim = 10 # Dimension of each point
min_distance = 0.1  # Minimum distance between points

# Generate points with minimum distance constraint
points = generate_points_with_min_distance(N, dim=dim, min_distance=min_distance, epsilon=1e2)

# Print the generated points
print("Generated points with at least 0.1 distance from each other:")
print(points)

# Verify sum and L2 norm of each vector
sums = np.sum(points, axis=1)
norms = np.linalg.norm(points, axis=1)
print("Sum of elements of each vector:")
print(sums)
print("L2 norm of each vector:")
print(norms)

# Check distances between all pairs of points
distances = np.linalg.norm(points[:, np.newaxis] - points[np.newaxis, :], axis=-1)
count_smaller_than_one = np.sum((distances < min_distance) & (distances > 0))
print(f"Number of distances smaller than {min_distance}: {count_smaller_than_one}")

np.save('points.npy', points)

Generated points with at least 0.1 distance from each other:
[[-31.33826662 -31.28721039 -31.2345177  ... -32.06274073 -31.73531808
  -31.34375326]
 [-31.85281149 -31.1945744  -31.43760657 ... -31.97213044 -32.3230901
  -31.91864034]
 [-32.00013072 -31.48192245 -31.91809843 ... -31.56348713 -31.60704374
  -31.58351641]
 ...
 [-31.91881767 -31.50834259 -31.55676506 ... -31.83502445 -31.53057906
  -31.47208682]
 [-31.58941042 -31.90824559 -31.30073287 ... -32.02464555 -31.40268807
  -32.10997222]
 [-31.74967161 -31.1896858  -31.99712621 ... -31.70305965 -32.04107823
  -31.75975012]]
Sum of elements of each vector:
[-316.52828971 -316.51963628 -316.53882977 -316.51341892 -316.53015897
 -316.53608228 -316.51850677 -316.5394636  -316.53702908 -316.53260061
 -316.53857607 -316.52678691 -316.51380271 -316.53674744 -316.53382319
 -316.53549708 -316.5351544  -316.53683689 -316.53547814 -316.53500652
 -316.53714653 -316.5258766  -316.53201558 -316.53715951 -316.53453752
 -316.51314999 -316.53674

In [411]:
import dis
from sturdy.utils.yiop import yiop_allocation_algorithm


new_point = np.random.randn(dim)
new_point -= np.mean(new_point) + 1e-10
new_point = new_point / np.linalg.norm(new_point) * (0.1 + 1e-10)

def disturb(synapse, x=[-0.02496461562494551, 0.051408458358368374, -0.06579014135927644, -0.01585099674629362, -0.01394016217903996, 0.01837961555707405, 0.0273245149569867, 0.0033646765988309965, 0.028239786989717828, -0.008171136574083209]):
    allocation = yiop_allocation_algorithm(synapse)
    allocation = {f'{i}': x[i] + allocation[str(i)] for i in range(10)}
    return allocation

assets_and_pools = generate_assets_and_pools()
yiop_allocation = yiop_allocation_algorithm(AllocateAssets(assets_and_pools=assets_and_pools))
disturb_vec = [-0.023303940802866446, 0.011722540235190102, 0.012532804543590058, 0.07148052115117533, -0.011637462120281968, 0.006821723380849581, -0.0049378838502735, -0.02901574112715599, 0.01808380148769488, -0.051746362953450975]
disturb_allocation = disturb(AllocateAssets(assets_and_pools=assets_and_pools), x=disturb_vec)
disturb_allocation

{'0': 0.20449575493313357,
 '1': 0.039607725886190104,
 '2': 0.27818336089759005,
 '3': 0.28629484204317535,
 '4': -0.011637462120281968,
 '5': 0.1903422958968496,
 '6': -0.0009321974592734995,
 '7': 0.145033232945844,
 '8': 0.01808380148769488,
 '9': 0.050528645430549024}

In [422]:

def negative_avoidance(X):
    X = np.array(X)
    Y = np.zeros_like(X)

    # Identify indices with negative and non-negative values in X
    negative_indices = np.where(X < 0)[0]
    positive_indices = np.where(X >= 0)[0]

    # Calculate the total negative sum
    total_negative_sum = -np.sum(X[negative_indices])

    # Calculate the proportionate redistribution of the negative sum
    positive_sum = np.sum(X[positive_indices])
    if positive_sum > 0:
        redistribution_factors = X[positive_indices] / positive_sum
        redistributed_values = total_negative_sum * redistribution_factors

        # Set Y values to offset negative values in X
        Y[negative_indices] = -X[negative_indices]
        Y[positive_indices] = -redistributed_values
    else:
        # If there are no positive values, we cannot redistribute; return an error
        raise ValueError("All values in X are negative or zero, cannot make X + Y non-negative.")

    return Y

result = {'0': 0.20449575493313357,
 '1': 0.039607725886190104,
 '2': 0.27818336089759005,
 '3': 0.28629484204317535,
 '4': -0.01637462120281968,
 '5': 0.1903422958968496,
 '6': -0.01009321974592734995,
 '7': 0.145033232945844,
 '8': 0.01808380148769488,
 '9': 0.050528645430549024}

vec = np.array(list(result.values()))
y = negative_avoidance(vec)
vec + y


array([0.20003204, 0.03874317, 0.2721112 , 0.28004563, 0.        ,
       0.18618752, 0.        , 0.14186746, 0.01768907, 0.04942571])

In [376]:

import pandas as pd


apys = {
    'yiop': [],
    'disturb': []
}
for i in range(1000):
    apys['yiop'].append(calc_strategy_apy(yiop_allocation, assets_and_pools))
    apys['disturb'].append(calc_strategy_apy(disturb_allocation, assets_and_pools))

df = pd.DataFrame(apys)

In [28]:
%load_ext autoreload
%autoreload 2
import numpy as np
from sturdy.pools import generate_assets_and_pools
from sturdy.protocol import AllocateAssets
from sturdy.utils.yiop import yiop_allocation_algorithm
from sturdy.plarism_cheater import PlarsimCheater


assets_and_pools = generate_assets_and_pools()

cheater = PlarsimCheater('sphere_points.npy')
allocation = yiop_allocation_algorithm(AllocateAssets(assets_and_pools=assets_and_pools))
results = cheater.generate(allocation)

distances = []
for vec_x in results:
    for vec_y in results:
        distance = np.linalg.norm(np.array(list(vec_x.values())) - np.array(list(vec_y.values())))
        if distance < 0.1:
            distances.append(distance)

len(distances)
apys_and_allocations = {
    i: {
        'apy': 0,
        'allocations': results[i]
    } for i in range(len(results))
}

axon_times = {k: 0 for k, _ in apys_and_allocations.items()}

similarity_matrix = {}
for miner_a, info_a in apys_and_allocations.items():
    _alloc_a = info_a["allocations"]
    alloc_a = np.array(
        list(format_allocations(_alloc_a, assets_and_pools).values())
    )
    similarity_matrix[miner_a] = {}
    for miner_b, info_b in apys_and_allocations.items():
        if miner_a != miner_b:
            _alloc_b = info_b["allocations"]
            alloc_b = np.array(
                list(format_allocations(_alloc_b, assets_and_pools).values())
            )
            similarity_matrix[miner_a][miner_b] = np.linalg.norm(alloc_a - alloc_b)

# Step 2: Apply penalties considering axon times
penalties = calculate_penalties(similarity_matrix, axon_times)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:37                                                                                   │
│                                                                                                  │
│   34 for miner_a, info_a in apys_and_allocations.items():                                        │
│   35 │   _alloc_a = info_a["allocations"]                                                        │
│   36 │   alloc_a = np.array(                                                                     │
│ ❱ 37 │   │   list(format_allocations(_alloc_a, assets_and_pools).values())                       │
│   38 │   )                                                                                       │
│   39 │   similarity_matrix[miner_a] = {}                                                         │
│   40 │   for miner_b, info_b in apys_and_allocations.items():                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'format_allocations' is not defined